In [26]:
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from skimage import io, color
import cv2
import random

In [27]:


def harris_corner_detection(img, sigma, img_pair_id, k=0.05, threshold_factor=1):
    """
    Perform Harris Corner Detection on an image.
    
    Parameters:
    img: np.array
        Input image.
    sigma: float
        Standard deviation for Gaussian window.
    img_pair_id: str
        ID for image pair, used in file output naming.
    k: float, optional
        Harris detector free parameter (default is 0.05).
    threshold_factor: float, optional
        Factor to adjust corner detection threshold (default is 1).
    
    Returns:
    corners_top: np.array
        Array of top corners detected.
    img_output: np.array
        Output image with corners drawn.
    """
    # Convert the image to grayscale and normalize
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255

    # Haar-like filter for x and y derivatives
    haar_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
    haar_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])

    # Apply convolution to get derivatives
    dx = cv2.filter2D(img_gray, -1, haar_x)
    dy = cv2.filter2D(img_gray, -1, haar_y)
    
    # Calculate products of derivatives
    dx2 = dx * dx
    dy2 = dy * dy
    dxy = dx * dy

    # Gaussian filter to smooth derivative products
    gauss_kernel_size = (int(4 * sigma + 1), int(4 * sigma + 1))
    dx2_sum = cv2.GaussianBlur(dx2, gauss_kernel_size, sigma)
    dy2_sum = cv2.GaussianBlur(dy2, gauss_kernel_size, sigma)
    dxy_sum = cv2.GaussianBlur(dxy, gauss_kernel_size, sigma)

    # Compute Harris corner response
    trace = dx2_sum + dy2_sum
    det = dx2_sum * dy2_sum - dxy_sum**2
    R = det - k * (trace**2)

    # Determine threshold for corner detection
    R_thresh = threshold_factor * np.mean(np.abs(R))

    # Non-maximum suppression for corner points
    corners = np.zeros_like(R)
    N = int(5 * sigma)  # Window size is proportional to sigma

    for y in range(N, img_gray.shape[0] - N):
        for x in range(N, img_gray.shape[1] - N):
            window = R[y - N:y + N + 1, x - N:x + N + 1]
            R_max = np.max(window)
            if R[y, x] == R_max and R_max > R_thresh:
                corners[y, x] = R_max

    # Get the top 100 corner points
    corners_top = np.argpartition(corners.flatten(), -100)[-100:]
    corners_top = np.vstack(np.unravel_index(corners_top, corners.shape)).T

    # Output image with corners highlighted
    img_output = img.copy()
    for point in corners_top:
        cv2.circle(img_output, tuple(point[::-1]), radius=4, color=(0, 0, 255), thickness=2)

    # Save the output image (flexible path for portability)
    output_filename = f'Harris_scale_{sigma}_{img_pair_id}.jpg'
    cv2.imwrite(output_filename, img_output)

    return corners_top, img_output




In [28]:
def ncc_metric(img1, img2, img1_corners, img2_corners, sigma, B, img_pair_id):
    """
    Calculate the Normalized Cross-Correlation (NCC) metric between two images.
    
    Parameters:
    img1: np.array
        First input image.
    img2: np.array
        Second input image.
    img1_corners: np.array
        Corner points from image 1.
    img2_corners: np.array
        Corner points from image 2.
    sigma: float
        Standard deviation for Gaussian window.
    B: int
        Border size for padding.
    img_pair_id: str
        Image pair ID for saving the output image.
    
    Returns:
    img12: np.array
        Output image with corners and NCC results drawn.
    """
    # Convert the images to grayscale and normalize
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255

    # Apply border padding to both images
    img1_border = cv2.copyMakeBorder(img1_gray, B, B, B, B, cv2.BORDER_CONSTANT, value=0)
    img2_border = cv2.copyMakeBorder(img2_gray, B, B, B, B, cv2.BORDER_CONSTANT, value=0)
    # Prepare RGB versions of the bordered images for visualization
    img1_RGB_border = cv2.copyMakeBorder(img1, B, B, B, B, cv2.BORDER_CONSTANT, value=0)
    img2_RGB_border = cv2.copyMakeBorder(img2, B, B, B, B, cv2.BORDER_CONSTANT, value=0)

    # Concatenate the images horizontally for display
    img12 = np.concatenate((img1_RGB_border, img2_RGB_border), axis=1)

    # Initialize NCC value array
    ncc_values = np.zeros((len(img1_corners), 2))

# Calculate NCC for each corner in img1 with each corner in img2
    for i, coord1 in enumerate(img1_corners):
        ncc = np.zeros((len(img2_corners), 2))
        
        for j, coord2 in enumerate(img2_corners):
            # Check if window stays within bounds for img1
            if coord1[1] + B > img1_border.shape[0] or coord1[0] + B > img1_border.shape[1]:
                continue
            
            # Check if window stays within bounds for img2
            if coord2[1] + B > img2_border.shape[0] or coord2[0] + B > img2_border.shape[1]:
                continue

            # Define the window region for both images
            img1_window = img1_border[coord1[1]:coord1[1] + B, coord1[0]:coord1[0] + B]
            img2_window = img2_border[coord2[1]:coord2[1] + B, coord2[0]:coord2[0] + B]

            # Compute the means of the windows
            m1, m2 = np.mean(img1_window), np.mean(img2_window)

            # Calculate numerator and denominator of NCC
            numer = np.sum((img1_window - m1) * (img2_window - m2))
            denom = np.sqrt(np.sum((img1_window - m1) ** 2) * np.sum((img2_window - m2) ** 2))
            
            # Store NCC value if denominator is not zero
            if denom != 0:
                ncc[j] = numer / denom


        # Find the maximum NCC value and the corresponding corner in img2
        if np.max(ncc[:, 0]) > 0.3:
            ncc_largest_idx = np.argmax(ncc[:, 0], axis=0)
            coord2_largest = img2_corners[ncc_largest_idx]
            
            # Visualize the corresponding points
            coord1_plot = coord1 + B  # Adjust for the border size
            # coord2_plot = coord2_largest + np.array([B + width, B])  # Adjust for concatenated image
            coord2_plot = coord2_largest + np.array([B*3 + img1_RGB_border.shape[1], B])
            random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.circle(img12, tuple(coord1_plot), radius=4, color=(0, 0, 255), thickness=2)
            cv2.circle(img12, tuple(coord2_plot), radius=4, color=(255, 0, 0), thickness=2)
            cv2.line(img12, tuple(coord1_plot), tuple(coord2_plot), color=random_color, thickness=1)
    
    # Save the output image
    output_filename = f'NCC_{img_pair_id}_scale_{sigma}.jpg'
    cv2.imwrite(output_filename, img12)
    print(f"Saved image as {output_filename}")




In [29]:

def SSD_metric(img1, img2, img1_corners, img2_corners, sigma, B, img_pair_id):
    # Convert images to grayscale and normalize
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255

    # Apply border padding to both images to handle boundary conditions
    img1_border = cv2.copyMakeBorder(img1_gray, B, B, B, B, cv2.BORDER_CONSTANT, value=0)
    img2_border = cv2.copyMakeBorder(img2_gray, B, B, B, B, cv2.BORDER_CONSTANT, value=0)

    # Prepare RGB versions of the bordered images for visualization
    img1_RGB_border = cv2.copyMakeBorder(img1, B, B, B, B, cv2.BORDER_CONSTANT, value=0)
    img2_RGB_border = cv2.copyMakeBorder(img2, B, B, B, B, cv2.BORDER_CONSTANT, value=0)

    # Concatenate the two images horizontally for easier visualization
    img12 = np.concatenate((img1_RGB_border, img2_RGB_border), axis=1)

    # Initialize SSD values for storing results
    SSD_shortest = np.zeros((len(img1_corners)))
    
    # Get the width of the first image (before concatenation)
    width = img1.shape[1]

    # Calculate SSD for each corner in img1 compared to all corners in img2
    for i, coord1 in enumerate(img1_corners):
        SSD = np.zeros((len(img2_corners)))

        for j, coord2 in enumerate(img2_corners):
            # Define the windows (patches) in both images based on the corners
            img1_window = img1_border[coord1[1]:coord1[1] + B, coord1[0]:coord1[0] + B]
            img2_window = img2_border[coord2[1]:coord2[1] + B, coord2[0]:coord2[0] + B]

            # Ensure both windows have the correct size before comparing
            if img1_window.shape == img2_window.shape and img1_window.size == B * B:
                # Compute the SSD metric between the windows
                SSD[j] = np.sum((img1_window - img2_window) ** 2)

        # Store the index of the best match in img2 based on the smallest SSD
        SSD_shortest[i] = np.argmin(SSD)

    # Visualize the SSD correspondence result
    for i, coord1 in enumerate(img1_corners):
        # Get the corresponding point in img2 with the minimum SSD
        coord2 = img2_corners[int(SSD_shortest[i])]

        # Adjust for borders and concatenation
        coord1_plot = coord1 + B  # Adjust for the border
        coord2_plot = coord2 + np.array([B + width, B])  # Adjust for the border and concatenation
        random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Draw circles at the corresponding points
        cv2.circle(img12, tuple(coord1_plot), radius=4, color=(0, 0, 255), thickness=2)  # Red circle for img1
        cv2.circle(img12, tuple(coord2_plot), radius=4, color=(255, 0, 0), thickness=2)  # Blue circle for img2
        cv2.line(img12, tuple(coord1_plot), tuple(coord2_plot), color=random_color, thickness=1)  # Green line

    # Save the output image showing the SSD correspondences
    output_filename = f'SSD_{img_pair_id}_scale_{sigma}.jpg'
    cv2.imwrite(output_filename, img12)
    print(f"Saved image as {output_filename}")

    # return img12


In [30]:
def sift_feature_matching(img1, img2, ratio_test=0.75):
    # Step 1: Initialize the SIFT detector
    sift = cv2.SIFT_create()

    # Step 2: Detect keypoints and compute descriptors for both images
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Step 3: Use BFMatcher to match descriptors
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # Step 4: Find the top two matches for each descriptor in img1
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Step 5: Apply ratio test to keep only good matches
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good_matches.append(m)

    # Step 6: Draw matches
    match_img = cv2.drawMatches(
        img1, keypoints1, img2, keypoints2, good_matches, None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    # Step 7: Return the image with matches
    return match_img, keypoints1, keypoints2, good_matches



In [31]:
# Example usage:
hovde_img_2 = cv2.imread('hovde_2.jpg')
hovde_img_3 = cv2.imread('hovde_3.jpg')
temple_img_1 = cv2.imread('temple_1.jpg')
temple_img_2 = cv2.imread('temple_2.jpg')

sigmas=[1.2, 1.6, 2.0]
for sigma in sigmas:
    # Hovde
    corners_hv_2, hovde_img_2_harr = harris_corner_detection(hovde_img_2, sigma=sigma, img_pair_id='hovde_2')
    corners_hv_3, hovde_img_3_harr = harris_corner_detection(hovde_img_3, sigma=sigma, img_pair_id='hovde_3')
    #  Temple
    corners_hv_4, temple_img_1_harr = harris_corner_detection(temple_img_1, sigma=sigma, img_pair_id='temple_1')
    corners_hv_5, temple_img_2_harr = harris_corner_detection(temple_img_2, sigma=sigma, img_pair_id='temple_2')

    # Evaluation
    # Temple
    ncc_metric(hovde_img_2, hovde_img_3, corners_hv_2, corners_hv_3, sigma=sigma, B=10, img_pair_id='pair1')
    SSD_metric(hovde_img_2, hovde_img_3, corners_hv_2, corners_hv_3, sigma=sigma, B=10, img_pair_id='pair1')
    # Hovde
    ncc_metric(temple_img_1, temple_img_2, corners_hv_4, corners_hv_5, sigma=sigma, B=10, img_pair_id='pair2')
    SSD_metric(temple_img_1, temple_img_2, corners_hv_4, corners_hv_5, sigma=sigma, B=10, img_pair_id='pair2')




Saved image as NCC_pair1_scale_1.2.jpg
Saved image as SSD_pair1_scale_1.2.jpg
Saved image as NCC_pair2_scale_1.2.jpg
Saved image as SSD_pair2_scale_1.2.jpg
Saved image as NCC_pair1_scale_1.6.jpg
Saved image as SSD_pair1_scale_1.6.jpg
Saved image as NCC_pair2_scale_1.6.jpg
Saved image as SSD_pair2_scale_1.6.jpg


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:294: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'cv::createGaussianKernels'


In [ ]:
# SIFT 
match_img, kp1, kp2, matches = sift_feature_matching(hovde_img_2, hovde_img_3)
# Save and show the result
cv2.imwrite('sift_matches.jpg', match_img)
